In [1]:
import pandas as pd
from scipy import spatial
from util import to_Cartesian,geo_to_cartesian,distToKM,kmToDIST
import geopandas as gpd
import numpy as np
from geopandas.tools import sjoin
import folium
from folium.plugins import MarkerCluster
import shapely
from shapely.geometry import Point
import unicodedata
import pysal as ps
from shapely import geometry
import sys
from folium import plugins
import string
import folium
from shapely import wkt, geometry
import json
from pprint import pprint
from openrouteservice import client, places
from openrouteservice import distance_matrix
from geopandas import GeoDataFrame
from shapely.geometry import Point
import osrm
from openrouteservice import geocoding

import re


#import osrm
#reload(sys)
#sys.setdefaultencoding('utf-8')

#Setup open-route service client
api_key = '58d904a497c67e00015b45fc07893889d08b4b6abcfc82938c195ec0'
clnt = client.Client(key=api_key)


# Make a new RequestConfig object
MyConfig = osrm.RequestConfig()
MyConfig.host = "http://router.project-osrm.org"  # Only change the host
osrm.RequestConfig.host = "router.project-osrm.org"
osrm.RequestConfig.host

#Read data
column_names = 'id;'.split(";")

all_data = pd.read_csv('./data/processed_data.csv',sep=",")

#Convert geo-coordinates to cartesian
all_data['x'], all_data['y'], all_data['z'] = geo_to_cartesian(all_data['long_wgs84'],all_data['lat_wgs84'])

stationArr = all_data[['lat_wgs84', 'long_wgs84']].as_matrix()

import geocoder

#https://geopy.readthedocs.io/en/stable/
from geopy.geocoders import Nominatim

addresses=[]
#Reverse geo code the address name
for i in range(len(all_data)):
    x = all_data['lat_wgs84'][i]
    y = all_data['long_wgs84'][i]
    #geolocator = Nominatim()
    #location = geolocator.reverse([x, y])
    address = all_data['address'][i]
    name = address.split('-')
    addresses.append(name[0].capitalize())
    #g = geocoder.google([y,x], method='reverse')
    #print(g.street)
    #all_data['name'] = clnt.reverse_geocode(location=(x, y))['features'][0]['properties']['name']
    
all_data['name'] = addresses
all_data.to_csv('./data/final_dataset.csv')
#locationlist = zip(all_data['long_wgs84'],all_data['lat_wgs84'])
locationlist = [[all_data['long_wgs84'][i],all_data['lat_wgs84'][i]] for i in range(len(all_data)) ]

False
False
True
True


In [2]:
from graph import Graph
g = {}

print(weights.weight(0,1))

graph = Graph(g)

print("Vertices of graph:")
print(graph.vertices())

print("Edges of graph:")
print(graph.edges())

print("Add vertex:")
graph.add_vertex("4")

print("Vertices of graph:")
print(graph.vertices())

print("Add an edge:")
graph.add_edge({2,3})

print("Vertices of graph:")
print(graph.vertices())

print("Edges of graph:")
print(graph.edges())

print('Adding an edge {"x","y"} with new vertices:')
graph.add_edge({1,3})
print("Vertices of graph:")
print(graph.vertices())
print("Edges of graph:")
print(graph.edges())
  

NameError: name 'weights' is not defined

In [3]:
from folium.plugins import MarkerCluster
from folium import IFrame

#Create SF basemap specifying map center, zoom level, and using the default OpenStreetMap tiles
map = folium.Map(location=[50.821658, 4.394886], zoom_start=15)


points=[]

#Folium bugs when using numpy -> use a list of lists as an alternative
stationArr = [[all_data['latitude'][i],all_data['longitude'][i],all_data['pm2.5'][i]] for i in range(len(all_data['latitude']))]
locationlist = [[all_data['longitude'][i],all_data['latitude'][i]] for i in range(len(all_data))]


#Create empty lists to contain the point coordinates and the point pop-up information
coords, popups = [], [] 


for point in range(0, len(all_data)):
    #t='{0},({1}): {2}'.format(area_of_circle, points, estimate(radius, points))
    address=all_data['address'][point].split('-')[0].decode('latin-1').capitalize()
  
    x=all_data['latitude'][point]
    y=all_data['longitude'][point]
    print(address,x,y)
    pm=all_data['pm2.5'][point]
    #text=u"<strong>{0}</strong><br>Lat: {1:.3f}<br>Long: {2:.3f}".format(address,x,y,pm)
    text = u"<strong>{0}</strong><br>Lat: {1:.3f}<br>Long: {2:.3f}".format(address, x, y)
    popup = folium.Popup(text, parse_html=True)
    #folium.CircleMarker(locationlist[point],popup='test').add_to(map)
    #folium.Marker([x,y],popup=popup).add_to(map)
    
    #Append lat and long coordinates to "coords" list
    coords.append([x,y])
    #Create a string of HTML code used in the IFrame popup
    #Join together the fields in "popup_field_list" with a linebreak between them
    #label = '<br>'.join([row[field] for field in popup_field_list])
    #Append an IFrame that uses the HTML string to the "popups" list 
    popups.append(IFrame(text, width = 300, height = 100))
        


#Create a Folium feature group for this layer, since we will be displaying multiple layers
pt_lyr = folium.FeatureGroup(name = 'pt_lyr')

#Add the clustered points of crime locations and popups to this layer
pt_lyr.add_children(MarkerCluster(locations = coords, popups = popups))

#Add this point layer to the map object
map.add_children(pt_lyr)  




max_amount = float(all_data['pm2.5'].max())
map.add_child(plugins.HeatMap(stationArr,
                                 radius=15,
                                 min_opacity=0.2,
                                 max_val=max_amount,
                                 blur=15, 
                                 max_zoom=10))

map.save('routing.html')

(u'Ferm\xe9e', 50.812488999999999, 4.3792029999999995)
(u'Biestebroeck / biestebroek ', 50.833424000000001, 4.3133780000000002)
(u'Ten reuken ', 50.804454999999997, 4.4263690000000002)
(u'Hall des sports ', 50.877037000000001, 4.3008470000000001)
(u'Hotel president/president hotel ', 50.863875, 4.358708)
(u'Abbaye de la cambre/abdij ter kameren ', 50.820042000000001, 4.3747850000000001)
(u'Forest national / vorst nationaal ', 50.810085000000001, 4.3248639999999998)
(u'Place st jean/sint', 50.845123999999998, 4.3514860000000004)
(u'Vanderkindere ', 50.81418, 4.3477209999999999)
(u'Atomium ', 50.895485000000001, 4.3413110000000001)
(u'Altitude cent / hoogte honderd ', 50.816524999999999, 4.3362120000000006)
(u'Reine fabiola / koningin fabiola ', 50.872475000000001, 4.3142659999999999)
(u'Chee de waterloo/waterloosesteenweg 1050', 50.798988000000001, 4.3755809999999995)
(u'Tractebel ', 50.856582000000003, 4.4309880000000001)
(u'Place bockstael/bockstaelplaats ', 50.878079999999997, 4.3479

(u'Geyskens ', 50.819546000000003, 4.4211980000000004)
(u'Athenee de jette/atheneum van jette ', 50.871192745100004, 4.3299247656400004)
(u'Place albert/plein ', 50.821117000000001, 4.3420079999999999)
(u'Stalle ', 50.795859999999998, 4.3205800000000005)
(u'Jardin massart ', 50.813290000000002, 4.4347510000000003)
(u'Gare du nord/noordstation ', 50.859181, 4.3594200000000001)
(u'Parnasse ', 50.837713999999998, 4.3691170000000001)
(u'Schuman ', 50.843359, 4.3799460000000003)
(u'Notre dame / onze lieve vrouw ', 50.870983000000003, 4.3975059999999999)
(u'Boileau ', 50.831679999999999, 4.4047689999999999)
(u'Clovis ', 50.850619999999999, 4.3811519999999993)
(u'Ulb plaine ', 50.818253000000006, 4.3957489999999995)
(u'Anneessens ', 50.844045000000001, 4.3450480000000002)
(u'Houffalize ', 50.865217999999999, 4.3777349999999995)
(u'Delacroix ', 50.845928000000001, 4.3238120000000002)
(u'Bois de la cambre ', 50.812179999999998, 4.3683209999999999)
(u'Masui ', 50.869296999999996, 4.3625699999999

(u'De paduwa ', 50.859127000000001, 4.4144059999999996)
(u'Geneve ', 50.861709999999995, 4.4031469999999997)
(u'Hansen', 50.831078000000005, 4.3985459999999996)
(u'Sainte marie/sinte', 50.857779999999998, 4.368144)
(u'Zaman forest national/zaman vorst nationaal ', 50.813586999999998, 4.324681)
(u'Centre culturel / cultureel centrum ', 50.816947999999996, 4.4265509999999999)
(u'Avenue paul hymanslaan', 50.843617000000002, 4.4253239999999998)
(u'Jules cesar ', 50.834705, 4.4216430000000004)
(u'Ferm\xe9e ', 50.797739, 4.3360830000000004)
(u'Jambline de meux ', 50.847158, 4.3932549999999999)
(u'Claessens ', 50.873084999999996, 4.3585900000000004)
(u'Midi fonsny/zuid fonsny ', 50.835698999999998, 4.3388260000000001)
(u'Ferm\xe9e ', 50.875209999999996, 4.3179540000000003)
(u'Beekant ', 50.854165999999999, 4.3229480000000002)
(u'Ribaucourt ', 50.860861999999997, 4.3391820000000001)
(u'Erasme ', 50.814820000000005, 4.2653400000000001)
(u'Kennis ', 50.867643000000001, 4.3858169999999994)
(u'Sai

/home/god/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:47: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.
/home/god/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:50: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.


In [8]:
from graph import Graph

g = {}
w = {}

graph = Graph()




# convert the 30 km to cartesian coordinates distance
#dist = kmToDIST(1)


# create the KD-tree using the 3D cartesian coordinates
coordinates = list(zip(all_data['x'], all_data['y'], all_data['z']))
tree = spatial.cKDTree(coordinates)


# convert the 30 km to cartesian coordinates distance
dist = kmToDIST(3)



# get all the points within 30 km from the reference point
#ix = tree.query_ball_point((x_ref, y_ref, z_ref), dist)


# get all the points within 30 km from the reference point
#ix = tree.query_ball_point((x_ref, y_ref, z_ref), dist)


all_data_array = all_data.as_matrix()
coord = all_data[['long_wgs84','lat_wgs84']].as_matrix()
nodes=[]
graph.add_nodes([i for i in range(len(all_data))])
for i in range(len(all_data)):
    x = all_data['long_wgs84'][i]
    y = all_data['lat_wgs84'][i]
    name = all_data['name'][i]
    print("Node : ",i,x,y,name)
    #Determine reference point
    x_ref, y_ref, z_ref = to_Cartesian(x,y)
    # get the cartesian distances from the 10 closest points
    dist, ix = tree.query((x_ref, y_ref, z_ref), 4)
    #graph.add_vertex(i)
    x = coord[i][0]
    y = coord[i][1]
    nodes.append(i)
    for j in range(1,len(dist)):
        index = ix[j]
        print(index,all_data_array[index][1],all_data_array[index][2],dist[j])
        distance = dist[j]
        #weights.add_weight(i, index,dist[j])
        x_target = coord[index][0]
        y_target = coord[index][1]
        folium.PolyLine([[y, x], [y_target, x_target]]).add_to(map)
        #folium.PolyLine([x,y]).add_to(map)
        #result = osrm.simple_route([x,y], [x_target, y_target],output='route', overview="full", geometry='wkt')
       
    #print(result['distance'])
        #print(result['LINESTRING'])
        #my_PolyLine=folium.PolyLine(locations=[[x,y],[x_target,y_target]])
        #map.add_child(my_PolyLine)
        graph.add_edge(i,index,distance,60)
        #print(dist[j],all_data.iloc(ix[j])['long_wgs84'])
        # print out the real distances in km
        #print(all_data.iloc[[ix[0]]])


        
print('done')
map
map.save('routing2.html')    
#print("Vertices of graph:")
#print(graph.vertices())
#print("Edges of graph:")
#print(graph.edges())

#Create weights


('Node : ', 0, 4.3792029999999995, 50.812488999999999, 'Ferm\xc9e')
(120, 50.814852, 4.376794, 16.123417492491029)
(133, 50.825590000000005, 4.379555, 27.37609143389966)
(309, 50.813724, 4.384205000000001, 31.969940774410716)
('Node : ', 1, 4.3133780000000002, 50.833424000000001, 'Biestebroeck / biestebroek ')
(306, 50.839554, 4.312311, 16.643556376770249)
(112, 50.843967, 4.313537, 26.110431987709124)
(338, 50.836864, 4.317321, 26.511818371005337)
('Node : ', 2, 4.4263690000000002, 50.804454999999997, 'Ten reuken ')
(264, 50.812325, 4.427962, 17.37909965648636)
(282, 50.816948, 4.426551, 22.478864139025006)
(246, 50.820244, 4.4300809999999995, 36.804200448894626)
('Node : ', 3, 4.3008470000000001, 50.877037000000001, 'Hall des sports ')
(177, 50.872169, 4.298216, 20.876104757073659)
(253, 50.866378999999995, 4.303281, 31.211222735268546)
(299, 50.864622, 4.297878, 36.955258151800066)
('Node : ', 4, 4.358708, 50.863875, 'Hotel president/president hotel ')
(27, 50.860652, 4.358116000000

(99, 50.848255, 4.314854, 16.948203008749729)
(46, 50.858058, 4.318689, 17.988211085942879)
(84, 50.862652000000004, 4.316644, 21.468738636379392)
('Node : ', 39, 4.4022879999999995, 50.821556999999999, 'Triomphe ')
(81, 50.818456, 4.403490000000001, 9.7392960332665623)
(247, 50.821735, 4.405622, 21.243693617630672)
(225, 50.810528000000005, 4.40231, 21.441547145489526)
('Node : ', 40, 4.3637969999999999, 50.827577000000005, 'Lesbroussart ')
(31, 50.834792, 4.364115, 15.824411543613239)
(186, 50.818283, 4.363568, 20.284187541109944)
(276, 50.833304, 4.366611, 21.806742787056834)
('Node : ', 41, 4.3621059999999998, 50.856714000000004, 'St')
(121, 50.850303000000004, 4.3607059999999995, 16.636268055588697)
(139, 50.859181, 4.35942, 17.948321895329762)
(29, 50.849358, 4.363589, 18.624184795765764)
('Node : ', 42, 4.4016739999999999, 50.880580000000002, 'Saint vincent / sint')
(161, 50.87693, 4.400722, 9.3533054752760965)
(203, 50.872002, 4.4032860000000005, 19.576879722379712)
(43, 50.868

(193, 50.83975, 4.385426, 23.936235286187884)
('Node : ', 80, 4.3661880000000002, 50.853248000000001, 'Botanique/kruidtuin ')
(280, 50.85778, 4.368144, 15.83551607550341)
(29, 50.849358, 4.363589, 18.585083211218791)
(258, 50.849761, 4.369356, 21.533555672605235)
('Node : ', 81, 4.4034900000000006, 50.818455999999998, 'Delta ')
(39, 50.821557, 4.4022879999999995, 9.7392960332665623)
(247, 50.821735, 4.405622, 14.985381760705813)
(225, 50.810528000000005, 4.40231, 17.122308978295408)
('Node : ', 82, 4.3092224000000003, 50.866795000000003, 'Chateau')
(55, 50.869032000000004, 4.309731, 6.4596494955453894)
(15, 50.874767, 4.308977, 19.993342076093747)
(204, 50.86002, 4.3075019999999995, 20.200689361436336)
('Node : ', 83, 4.3451120000000003, 50.890521999999997, 'Centenaire / eeuwfeest ')
(217, 50.885884999999995, 4.342003, 22.489350809514782)
(265, 50.881889, 4.347448, 24.681213696747683)
(9, 50.895485, 4.341311, 26.769406182973356)
('Node : ', 84, 4.3166440000000001, 50.862652000000004, '

('Node : ', 120, 4.3767940000000003, 50.814852000000002, 'Ceres ')
(0, 50.812489, 4.3792029999999995, 16.123417492491029)
(5, 50.820042, 4.374785, 16.825176376373289)
(195, 50.823986, 4.3737900000000005, 27.142674313408389)
('Node : ', 121, 4.3607059999999995, 50.850303000000004, 'Musee de la bd/stripmuseum ')
(255, 50.846319, 4.3622440000000005, 13.12028514009253)
(179, 50.843613, 4.360628, 14.692288662572073)
(41, 50.856714000000004, 4.362106, 16.636268055588697)
('Node : ', 122, 4.347791, 50.827954999999996, 'Place morichar / moricharplein ')
(59, 50.825749, 4.345165, 17.469636350013978)
(249, 50.829985, 4.344557, 21.117856311707985)
(167, 50.837912, 4.347421, 22.752934644267317)
('Node : ', 123, 4.3590309999999999, 50.827652, 'Defacqz ')
(200, 50.824048, 4.359896, 9.6620911319199703)
(273, 50.833079999999995, 4.35743, 15.743901827917201)
(320, 50.821567, 4.356508, 20.966173579150897)
('Node : ', 124, 4.3244769999999999, 50.876195000000003, 'Maison communale de jette / jette gemeent

(43, 50.868790000000004, 4.399587, 17.494901822720877)
(203, 50.872002, 4.4032860000000005, 18.941788814626825)
('Node : ', 162, 4.3963859999999997, 50.851240999999995, 'Plasky ')
(119, 50.855629, 4.394807, 13.305754118009578)
(153, 50.854654, 4.3984510000000006, 14.780321550954689)
(286, 50.847158, 4.393255, 21.537939065149914)
('Node : ', 163, 4.4184970000000003, 50.793682000000004, 'Coin du balai ')
(184, 50.798344, 4.419014, 9.2060006065232436)
(226, 50.814958000000004, 4.416987000000001, 40.521477618173158)
(132, 50.802675, 4.412073, 44.230515536598674)
('Node : ', 164, 4.3739669999999995, 50.889603000000001, 'Hembeek ')
(44, 50.881588, 4.373817, 16.983526953301208)
(37, 50.896278, 4.372427, 17.218366253040941)
(347, 50.880887, 4.374248000000001, 18.515117739183953)
('Node : ', 165, 4.357532, 50.867708, 'Jcdecaux ')
(4, 50.863875, 4.358708, 11.309259108680994)
(287, 50.873084999999996, 4.35859, 13.664018452752794)
(27, 50.860652, 4.358116000000001, 16.044489663179565)
('Node : ', 

(85, 50.848701, 4.3560550000000005, 7.945180185104082)
(75, 50.846331, 4.355193, 8.4637745601797736)
(259, 50.843556, 4.3550059999999995, 10.159442501932794)
('Node : ', 199, 4.3798859999999999, 50.872591999999997, 'Demolder ')
(269, 50.867955, 4.379576999999999, 9.8460280527183457)
(341, 50.877501, 4.380315, 10.561957229124822)
(147, 50.865218, 4.3777349999999995, 20.60133486758232)
('Node : ', 200, 4.359896, 50.824047999999998, 'Chatelain/kastelein ')
(123, 50.827652, 4.359031, 9.6620911319199703)
(320, 50.821567, 4.356508, 22.270213886877986)
(273, 50.833079999999995, 4.35743, 25.379761768976632)
('Node : ', 201, 4.3095910000000002, 50.851157999999998, 'Dubois ')
(157, 50.847649, 4.307714, 14.837012467198537)
(261, 50.854789000000004, 4.312201, 18.926852373368032)
(155, 50.854824, 4.306209, 23.425363933074845)
('Node : ', 202, 4.388191, 50.869764000000004, 'Aeropolis ')
(33, 50.877125, 4.3875150000000005, 15.561770629112576)
(293, 50.867643, 4.385816999999999, 15.729518239655404)
(2

(335, 50.803791, 4.333321, 29.110150114856232)
(77, 50.796471000000004, 4.328415000000001, 29.484786897224591)
('Node : ', 238, 4.4152459999999998, 50.855185999999996, 'Carene ')
(277, 50.859127, 4.414406, 9.1011646770869437)
(93, 50.849988, 4.417064, 15.08676961971125)
(208, 50.845365, 4.41788, 24.785692112441627)
('Node : ', 239, 4.3837219999999997, 50.845816999999997, 'Archimede / archimedes ')
(36, 50.841770000000004, 4.384180000000001, 8.8138417026309277)
(193, 50.83975, 4.385426, 16.514830037703025)
(73, 50.853972999999996, 4.38374, 16.772242895531839)
('Node : ', 240, 4.2959389999999997, 50.842666999999999, 'Peterbos ')
(242, 50.846874, 4.29817, 17.860249303573497)
(322, 50.837326000000004, 4.293932, 18.810169002797441)
(323, 50.832565, 4.294311, 28.068971351447168)
('Node : ', 241, 4.4033860000000002, 50.856208000000002, 'Jules lebrun ')
(278, 50.861709999999995, 4.403147, 10.772141648705269)
(58, 50.859243, 4.401022, 16.176160776128157)
(185, 50.867241, 4.40287, 21.64446835428

('Node : ', 276, 4.3666109999999998, 50.833303999999998, 'Fernand cocq ')
(31, 50.834792, 4.364115, 16.225573430130584)
(174, 50.829631, 4.3692, 18.289888260052511)
(140, 50.837714, 4.369117, 18.572898344585326)
('Node : ', 277, 4.4144059999999996, 50.859127000000001, 'De paduwa ')
(238, 50.855185999999996, 4.415246, 9.1011646770869437)
(93, 50.849988, 4.417064, 24.009321605173536)
(221, 50.862808, 4.410791000000001, 24.049907306687473)
('Node : ', 278, 4.4031469999999997, 50.861709999999995, 'Geneve ')
(241, 50.856208, 4.403386, 10.772141648705269)
(185, 50.867241, 4.40287, 10.873022451640585)
(58, 50.859243, 4.401022, 14.363842253873178)
('Node : ', 279, 4.3985459999999996, 50.831078000000005, 'Hansen')
(314, 50.826603000000006, 4.397117, 12.676519636584894)
(183, 50.839217, 4.398015, 16.374360453710157)
(329, 50.835797, 4.401599, 21.532832873861285)
('Node : ', 280, 4.368144, 50.857779999999998, 'Sainte marie/sinte')
(80, 50.853248, 4.366188, 15.83551607550341)
(65, 50.863872, 4.366

(234, 50.863388, 4.386222, 12.786250768385957)
(73, 50.853972999999996, 4.38374, 20.207596174172991)
('Node : ', 317, 4.3736309999999996, 50.795704000000001, 'Prince de ligne / prince lijn ')
(12, 50.798988, 4.3755809999999995, 14.227307881413473)
(251, 50.80534, 4.372123, 22.589592622960499)
(326, 50.815019, 4.371982, 42.313299555355044)
('Node : ', 318, 4.3329240000000002, 50.851562000000001, 'Auberge de jeunesse ')
(60, 50.8572655241, 4.33236837869, 13.926874492956618)
(35, 50.859608, 4.33409, 20.363773906767335)
(270, 50.849215, 4.329184, 24.468700474791)
('Node : ', 319, 4.3928310000000002, 50.801434, 'Gare de boondaelstation ')
(231, 50.807903, 4.392755999999999, 12.95750985929797)
(109, 50.816029, 4.391006, 31.514336273557515)
(118, 50.806271, 4.387673, 34.27919444904488)
('Node : ', 320, 4.3565079999999998, 50.821567000000002, 'Page')
(344, 50.827399, 4.355281, 15.141040300015391)
(207, 50.817392, 4.354105000000001, 17.911517661763941)
(123, 50.827652, 4.359031, 20.966173579150

In [26]:
vub_triomphe = [50.822, 4.402]
ulb = [50.814,4.384]


from deap import base, creator
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

import random
from deap import tools

IND_SIZE = 10

toolbox = base.Toolbox()
toolbox.register("attribute", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attribute, n=IND_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


def evaluate(individual):
    return sum(individual),

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)


def main():
    pop = toolbox.population(n=50)
    CXPB, MUTPB, NGEN = 0.5, 0.2, 40
    print(pop)
    # Evaluate the entire population
    fitnesses = map(toolbox.evaluate, pop)
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    for g in range(NGEN):
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = map(toolbox.clone, offspring)

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # The population is entirely replaced by the offspring
        pop[:] = offspring

    return pop

main()


[[0.47995806088203896, 0.7308618918901972, 0.7902532743656256, 0.07657646556941888, 0.8087663559882747, 0.5596057695471273, 0.24339578401674145, 0.953495708293858, 0.9055069208704515, 0.1485906364324111], [0.8702155443728721, 0.9298522591360014, 0.8932191271044511, 0.7375678410800153, 0.7607383415534863, 0.06085486797358963, 0.3204759769113191, 0.7169001971071332, 0.07214813692420885, 0.2367278722760786], [0.33020946013720454, 0.8579144909894331, 0.09170763391006143, 0.9153189455512936, 0.1783693860696831, 0.41153738757992386, 0.6223365892178974, 0.2594536102729934, 0.2159974055976226, 0.5573244710491692], [0.6084758086648272, 0.9250971987604478, 0.8378096578721431, 0.835057756251637, 0.12716375433492932, 0.4915828034024894, 0.011170548289705873, 0.21299599478699638, 0.7032245203462086, 0.5926976114861042], [0.5531702102186741, 0.12810048812295727, 0.5189985137907963, 0.8604640083013484, 0.5233798309077942, 0.5760867289661427, 0.7336010801519971, 0.8572861163216866, 0.7891625533519829,

TypeError: 'Map' object is not callable

In [5]:
#print(weights.)

folium.PolyLine([[50.840499999999999, 4.356878], [50.875826000000004,  4.32992476564]], color="red", weight=5, opacity=0.8).add_to(map)
folium.PolyLine([[50.821658, 4.394886], [50.81563, 4.9]], color="red", weight=5, opacity=0.8).add_to(map)


map
map.save('routing6.html')   

In [6]:
from openrouteservice import distance_matrix



request = {'locations': coord[0:10].tolist(),
           'profile': 'driving-car',
           'metrics': ['duration','distance']}
    
pubs_matrix = clnt.distance_matrix(**request)
print("Calculated {}x{} routes.".format(len(pubs_matrix['durations']),len(pubs_matrix['durations'][0])))
print(pubs_matrix)

Calculated 10x10 routes.
{u'info': {u'engine': {u'build_date': u'2018-03-26T14:15:39Z', u'version': u'4.5.0'}, u'attribution': u'openrouteservice.org | OpenStreetMap contributors', u'service': u'matrix', u'timestamp': 1525858010603, u'query': {u'profile': u'driving-car', u'units': u'm'}, u'osm_file_md5_hash': u'0d24b3ea819c14fd2072c0dc5effaf7e'}, u'distances': [[0, 6146.4, 5077.41, 11030.03, 7328.6, 1384.42, 5078.47, 4613.91, 2868.67, 11270.46], [6155.13, 0, 10854.87, 6636.04, 5371.81, 5468.17, 4338.87, 3446.88, 4843.52, 8317.45], [4799.02, 11002.94, 0, 15032.09, 10809.15, 5555.62, 9571.31, 9538.14, 7257.37, 14751], [11016.7, 6645.97, 15294.68, 0, 5957.27, 10253.98, 9954.43, 7026.92, 9540.61, 8562.71], [7394.52, 5797.61, 11268.27, 5856.17, 0, 6352.66, 7286.9, 3786.89, 6873.08, 4482.27], [1507.7, 5551.85, 5306.01, 10435.48, 6121.53, 0, 4483.92, 3808.24, 2553.85, 10063.39], [4826.57, 4421.31, 9414.58, 10038.59, 7306.49, 4334.87, 0, 4762.49, 2392.36, 10884.84], [4621.79, 3645.5, 9321.53, 

In [14]:

def getDistance(from_id, to_id):
    return int(pubs_matrix['durations'][from_id][to_id])
print(getDistance(0,1))

473


In [ ]:
shapely.speedups.enable()




bounded_area = [[50.809956 ,4.375437],[50.809956 ,4.380227],[50.825817,4.404967],[50.825817 ,4.403449]]


#Sort dataframe by coordinates
#all_data = all_data.sort_values(['latitude', 'longitude'], ascending=[True, True])


#Reset the indexes of dataframe after sorting
#all_data = all_data.reset_index(drop=True)

#
#all_data = all_data[0:20]
locations = all_data[['lat_wgs84', 'long_wgs84']]
locations = all_data[['long_wgs84', 'lat_wgs84']]
locations = all_data[['longitude', 'latitude']]
locationlist = locations.values.tolist()


wkt_str = 'Polygon ((13.43926404 52.48961046, 13.42040115 52.49586382, 13.42541101 52.48808523, 13.42368155 52.48635829, 13.40788599 52.48886084, 13.40852944 52.487142, 13.40745989 52.48614988, 13.40439187 52.48499746, 13.40154731 52.48500125, 13.40038591 52.48373202, 13.39423818 52.4838664, 13.39425346 52.48577149, 13.38629096 52.48582648, 13.38626853 52.48486362, 13.3715694 52.48495055, 13.37402099 52.4851697, 13.37416365 52.48771105, 13.37353615 52.48798191, 13.37539925 52.489432, 13.37643416 52.49167597, 13.36821531 52.49333093, 13.36952826 52.49886974, 13.37360623 52.50416333, 13.37497726 52.50337776, 13.37764916 52.5079675, 13.37893813 52.50693045, 13.39923153 52.50807711, 13.40022883 52.50938108, 13.40443425 52.50777471, 13.4052848 52.50821063, 13.40802944 52.50618019, 13.40997081 52.50692569, 13.41152096 52.50489127, 13.41407284 52.50403794, 13.41490921 52.50491634, 13.41760145 52.50417013, 13.41943091 52.50564912, 13.4230412 52.50498109, 13.42720031 52.50566607, 13.42940229 52.50857222, 13.45335235 52.49752496, 13.45090795 52.49710803, 13.44765912 52.49472124, 13.44497623 52.49442276, 13.43926404 52.48961046))'

aoi_geom = wkt.loads(wkt_str) # load geometry from WKT string

aoi_coords = list(aoi_geom.exterior.coords) # get coords from exterior ring
aoi_coords = [(y,x) for x,y in aoi_coords] # swap (x,y) to (y,x). Really leaflet?!
aoi_centroid = aoi_geom.centroid # Kreuzberg center for map center

from openrouteservice import client, places




gdf = gpd.GeoDataFrame(all_data.drop(['latitude', 'longitude'], axis=1),
                                crs={'init': 'epsg:4326'},
                                geometry=[shapely.geometry.Point(xy) for xy in zip(all_data.latitude, all_data.longitude)])




#print(result)
from scipy import spatial


  
import math
from collections import namedtuple

def haversine_distance(origin, destination):
  """ Haversine formula to calculate the distance between two lat/long points on a sphere """

  radius = 6371 # FAA approved globe radius in km

  dlat = math.radians(destination.lat-origin.lat)
  dlon = math.radians(destination.lng-origin.lng)
  a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(origin.lat)) \
      * math.cos(math.radians(destination.lat)) * math.sin(dlon/2) * math.sin(dlon/2)
  c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
  d = radius * c

  # Return distance in km
  return int(math.floor(d))

LatLng = namedtuple('LatLng', 'lat, lng')


#for origin_coordinates in locationlist:
#    origin = LatLng(origin_coordinates[0], origin_coordinates[1]) # London
#    for destination_coordinates in locationlist:
#        destination = LatLng(destination_coordinates[0], destination_coordinates[1]) # Athens
     
# convert the grid points and reference points to cartesian coordinates

# convert the grid points and reference points to cartesian coordinates
#x, y, z = zip(*map(to_Cartesian, lats_1d, lons_1d))

all_data['x'], all_data['y'], all_data['z'] = zip(*map(to_Cartesian, all_data['lat_wgs84'], all_data['long_wgs84']))


In [ ]:
from math import *

def to_Cartesian(lat, lng):
    '''
    function to convert latitude and longitude to 3D cartesian coordinates
    '''
    R = 6371 # radius of the Earth in kilometers

    x = R * cos(lat) * cos(lng)
    y = R * cos(lat) * sin(lng)
    z = R * sin(lat)
    return x, y, z

def deg2rad(degree):
    '''
    function to convert degree to radian
    '''
    rad = degree * 2*np.pi / 360
    return(rad)

def rad2deg(rad):
    '''
    function to convert radian to degree
    '''
    degree = rad/2/np.pi * 360
    return(degree)

def distToKM(x):
    '''
    function to convert cartesian distance to real distance in km
    '''
    R = 6371 # earth radius
    gamma = 2*np.arcsin(deg2rad(x/(2*R))) # compute the angle of the isosceles triangle
    dist = 2*R*sin(gamma/2) # compute the side of the triangle
    return(dist)

def kmToDIST(x):
    '''
    function to convert real distance in km to cartesian distance 
    '''
    R = 6371 # earth radius
    gamma = 2*np.arcsin(x/2./R) 
    
    dist = 2*R*rad2deg(sin(gamma / 2.))
    return(dist)

In [ ]:
from math import cos, asin, sqrt

def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a))

def closest(data, v):
    return min(data, key=lambda p: distance(v['lat'],v['lon'],p['lat'],p['lon']))

tempDataList = [{'lat': 39.7612992, 'lon': -86.1519681}, 
                {'lat': 39.762241,  'lon': -86.158436 }, 
                {'lat': 39.7622292, 'lon': -86.1578917}]

v = {'lat': 39.7622290, 'lon': -86.1519750}
print(closest(tempDataList, v))

In [ ]:
for origin_coordinates in locationlist:
    origin = LatLng(origin_coordinates[0], origin_coordinates[1]) # London
    for destination_coordinates in locationlist:
        destination = LatLng(destination_coordinates[0], destination_coordinates[1]) # Athens
        result = osrm.simple_route(origin, destination)
 


In [ ]:
from scipy import spatial
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

plt.style.use('seaborn-poster')

# define map ranges
llat = 50.821658
ulat = 60
llon = 4.394886
ulon = 5

# generate grid points
lats = np.arange(llat, ulat, 0.1)
lons = np.arange(llon, ulon, 0.1)
lons, lats = np.meshgrid(lons, lats)

# flatten the locations
lons_1d = lons.flatten()
lats_1d = lats.flatten()

# reference point as the center of the map
lat_0 = (llat + ulat) / 2.
lon_0 = (llon + ulon) / 2.

coordinates = [[lons_1d[i],lats_1d[i]] for i in range(len(lats))]
print(coordinates)

map = folium.Map(location=[50.821658, 4.394886], zoom_start=15)





In [ ]:






#map.save('main.html')

In [ ]:


from scipy import spatial
tree = spatial.KDTree(locationlist)

for coordinates in locationlist:
    x = coordinates[0]
    y = coordinates[1]
    result = tree.query(coordinates)
    print(result)
#tree = spatial.KDTree(locationlist)
#tree.query([(21,21)])

In [ ]:
result = osrm.simple_route(
                      [21.0566163803209,42.004088575972], [20.9574645547597, 41.5286973392856],
                      output='route', overview="full", geometry='wkt')
print(result)

In [ ]:
locationlist = locationlist[0:10]

from openrouteservice import distance_matrix


request = {'locations': locationlist,
           'profile': 'driving-hgv',
           'metrics': ['duration']}
    
pubs_matrix = clnt.distance_matrix(**request)
print(pubs_matrix)
print("Calculated {}x{} routes.".format(len(pubs_matrix['durations']),len(pubs_matrix['durations'][0])))

In [ ]:
o,r, c = osrm.table(locationlist, output='dataframe')
print(o)

In [ ]:
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

def getDistance(from_id, to_id):
    return int(pubs_matrix['durations'][from_id][to_id])

tsp_size = len(locationlist)
num_routes = 10
start = 0 # arbitrary start location
coords_aoi = [(y,x) for x,y in aoi_coords] # swap (x,y) to (y,x)

optimal_coords = []

if tsp_size > 0:
    routing = pywrapcp.RoutingModel(tsp_size, num_routes, start)
    search_parameters = pywrapcp.RoutingModel.DefaultSearchParameters()

    # Create the distance callback, which takes two arguments (the from and to node indices)
    # and returns the distance between these nodes.
    dist_callback = getDistance
    routing.SetArcCostEvaluatorOfAllVehicles(dist_callback)
    # Solve, returns a solution if any.
    assignment = routing.SolveWithParameters(search_parameters)
    if assignment:
        # Total cost of the 'optimal' solution.
        print("Total duration: " + str(round(assignment.ObjectiveValue(), 3) / 60) + " minutes\n")
        index = routing.Start(start) # Index of the variable for the starting node.
        route = ''
#         while not routing.IsEnd(index):
        for node in range(routing.nodes()):
            optimal_coords.append(locationlist[routing.IndexToNode(index)])
            #route += str(pubs_addresses[routing.IndexToNode(index)]) + ' -> '
            index = assignment.Value(routing.NextVar(index))
        #route += str(pubs_addresses[routing.IndexToNode(index)])
        optimal_coords.append(locationlist[routing.IndexToNode(index)])
        print(optimal_coords)

In [ ]:
from openrouteservice import directions
import os.path

def style_function(color):
    return lambda feature: dict(color=color,
                              weight=3,
                              opacity=1)




folium.features.GeoJson(data=optimal_route,
                        name='Optimal Bar Crawl',
                        style_function=style_function('#6666ff'),
                       overlay=True).add_to(map)

map.add_child(folium.map.LayerControl())
map

In [ ]:
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

df['dist'] = \
    haversine_np(df.LONG.shift(), df.LAT.shift(),
                 df.loc[1:, 'LONG'], df.loc[1:, 'LAT'])
    
    
    

In [ ]:
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

def getDistance(from_id, to_id):
    return int(pubs_matrix['durations'][from_id][to_id])

tsp_size = len(locationlist[0:2])
num_routes = 1
start = 0 # arbitrary start location
coords_aoi = [(y,x) for x,y in aoi_coords] # swap (x,y) to (y,x)

optimal_coords = []

if tsp_size > 0:
    routing = pywrapcp.RoutingModel(tsp_size, num_routes, start)
    search_parameters = pywrapcp.RoutingModel.DefaultSearchParameters()

    # Create the distance callback, which takes two arguments (the from and to node indices)
    # and returns the distance between these nodes.
    dist_callback = getDistance
    routing.SetArcCostEvaluatorOfAllVehicles(dist_callback)
    # Solve, returns a solution if any.
    assignment = routing.SolveWithParameters(search_parameters)
    if assignment:
        # Total cost of the 'optimal' solution.
        print("Total duration: " + str(round(assignment.ObjectiveValue(), 3) / 60) + " minutes\n")
        index = routing.Start(start) # Index of the variable for the starting node.
        route = ''
#         while not routing.IsEnd(index):
        for node in range(routing.nodes()):
            optimal_coords.append(pubs_coords[routing.IndexToNode(index)])
            print(pubs_addresses[routing.IndexToNode(index)])
            route += str(pubs_addresses[routing.IndexToNode(index)]) + ' -> '
            index = assignment.Value(routing.NextVar(index))
        route += str(pubs_addresses[routing.IndexToNode(index)])
        optimal_coords.append(pubs_coords[routing.IndexToNode(index)])
        print("Route:\n" + route)

In [ ]:

import random
from deap import creator, base, tools, algorithms

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=100)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evalOneMax(individual):
    return sum(individual),

toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

population = toolbox.population(n=300)

NGEN=40
for gen in range(NGEN):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.1)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    population = toolbox.select(offspring, k=len(population))
top10 = tools.selBest(population, k=10)

print(top10)

In [ ]:
platypus


## Data Prep
In this section I create two GeoDataFrames: one of crime points and one of census tract boundaries with crime densities. Both of these will then be plotted on a web map as separate layers.

### Read in Crime Data and Create a GeoDataFrame
First I read in a CSV file of San Francisco Police Incidents for the current year into a Pandas DataFrame. I downloaded the raw data from the San Francisco [Open Data Portal](https://data.sfgov.org/). Because there are so many crime incidents, I select a subset of the data: crimes in the "assault" category that were committed in the last 10 days. As shown below, this leaves me with 329 police incidents. 


In [ ]:
#Read in CSV file specifying date field and encoding. Sort by date
all_crime = pd.read_csv('SFPD_Incidents_-_Current_Year__2016_.csv', parse_dates = ['Date'],\
                        encoding = 'utf-8').sort_values('Date').reset_index(drop=True)

#Clean up the encoding on the Crime Description field 
all_crime.Descript = all_crime.Descript.apply(lambda x: unicodedata.normalize("NFKD", x))

#Create a field that contains a string representation of the date, for later use
all_crime['DateStr'] = all_crime.Date.apply(lambda x: x.strftime("%Y-%m-%d"))

#Identify those crimes that are categorized as assaults
is_assault = all_crime.Category == 'ASSAULT' 

#Identify those crimes that were committed in the most recent 10 days of the dataset
recent = all_crime.Date.isin(all_crime.Date.unique()[-10:]) 

#Subset the data to get assaults commited within the last 10 days
assaults = all_crime[is_assault & recent].drop_duplicates('IncidntNum').reset_index(drop = True)
assaults = assaults[['IncidntNum', 'Descript', 'DateStr', 'Time', 'Address','X', 'Y']]
print '{} assaults in the last 10 days'.format(str(len(assaults)))

I now want to convert the assault data Pandas DataFrame to a GeoPandas GeoDataFrame (a spatial version of the former). The raw crime data comes with lat/long coordinates, which I use these to create Shapely point geometry objects (these are the values in the "geometry" field for each record in a GeoDataFrame). I specify the spatial reference system as ESPG 4326 which represents the standard WGS84 coordinate system.

In [ ]:
#First create a GeoSeries of crime locations by converting coordinates to Shapely geometry objects
#Specify the coordinate system ESPG4326 which represents the standard WGS84 coordinate system
assault_geo = gpd.GeoSeries(assaults.apply(lambda z: Point(z['X'], z['Y']), 1),crs={'init': 'epsg:4326'})

#Create a geodataframe from the pandas dataframe and the geoseries of shapely geometry objects
assaults = gpd.GeoDataFrame(assaults.drop(['X', 'Y'], 1), geometry=assault_geo)
print assaults.head()

### Calculate Census Tract Crime Density
Next, I read in a Shapefile of San Francisco Census Tracts, which I also downloaded from the SF Open Data Portal. With GeoPandas, I can read a Shapefile directly into Python really easily. Then in one line of code, I spatially join census tracts to assaults (determine the census tract of each assault), and generate counts of assaults per census tract. Note that I use the ```to_crs``` function to convert assaults to the same coordinate system as Census Tracts (EPSG 3310) prior to spatially joining them.

Lastly, I calculate the number of assaults per square mile, which is the metric that I'm interested in plotting.

In [ ]:
#Read tracts shapefile into GeoDataFrame
tracts = gpd.read_file('sf_census_tracts.shp').set_index('CTFIPS10')

#Generate Counts of Assaults per Census Tract
#Spatially join census tracts to assaults (after projecting) and then group by Tract FIPS while counting the number of crimes
tract_counts = gpd.tools.sjoin(assaults.to_crs(tracts.crs), tracts.reset_index()).groupby('CTFIPS10').size()

#Calculate Assault Density, converting square meters to square miles.
tracts['AssaultsPSqMi'] = (tract_counts/(tracts.geometry.area*3.86102e-7)).fillna(0)
tracts = tracts.reset_index()
print tracts.head()

## Using Folium to Plot Data
The general approach I take here is to first create a Folium basemap and then add two layers to it: (1) a choropleth of census tracts, symbolized crime density, and (2) crime point locations. I write a separate function to plot each of these two layers, each of which takes a GeoDataFrame as its input. Folium takes unprojected lat/long coordinates for all of its plotting, so I make sure to convert all of my projected GeoDataFrames to WGS84 within the plotting functions.

###  Choropleth Layer of Tract Crime Density
As its inputs, my choropleth function takes a Folium map object, a GeoDataFrame, the name of the feature ID field, and the name of the field whose values will be symbolized. 

Leaflet uses GeoJSON objects to plot vector geometries (GeoJSON is a data format that is used to represent geographical features along with their non-spatial attributes). GeoPandas has a ```to_json``` method which I use to convert the GeoDataFrame to GeoJSON to be used as one of the inputs to Folium's choropleth function. I also specify the id field, which is used to link the geometry in the GeoJSON with the data in the GeoDataFrame.

The function also takes optional parameters for fill color, fill opacity, line opacity, number of classifiers, and classification scheme. All of these have default values if not specified. Folium / Leaflet uses the [Color Brewer](http://colorbrewer2.org/#type=sequential&scheme=YlOrRd&n=5) sequential color schemes, which can easily be specified to view different combinations. 

Lastly, I allow the user to specify the number of classes and the classification scheme. At this point, Folium has limited map classification options, so I instead use Pysal's choropleth [map classfication module](http://pysal.readthedocs.io/en/latest/library/esda/mapclassify.html) to provide some basic classification options. My function defaults to "Fisher_Jenks", but also has options for "Equal_Interval", and "Quantiles". 

Below, I first create a basemap that is centered in San Francisco, and then I run my function on this basemap specifying the Census Tract ID Field as well as the field I want to symbolize on (Assaults Per Square Mile).

In [ ]:
#Create SF basemap specifying map center, zoom level, and using the default OpenStreetMap tiles
crime_map = folium.Map([37.7556, -122.4399], zoom_start = 12)

def add_choropleth(mapobj, gdf, id_field, value_field, fill_color = 'YlOrRd', fill_opacity = 0.6, 
                    line_opacity = 0.2, num_classes = 5, classifier = 'Fisher_Jenks'):
    #Allow for 3 Pysal map classifiers to display data
    #Generate list of breakpoints using specified classification scheme. List of breakpoint will be input to choropleth function
    if classifier == 'Fisher_Jenks':
        threshold_scale=ps.esda.mapclassify.Fisher_Jenks(gdf[value_field], k = num_classes).bins.tolist()
    if classifier == 'Equal_Interval':
        threshold_scale=ps.esda.mapclassify.Equal_Interval(gdf[value_field], k = num_classes).bins.tolist()
    if classifier == 'Quantiles':
        threshold_scale=ps.esda.mapclassify.Quantiles(gdf[value_field], k = num_classes).bins.tolist()
    
    #Convert the GeoDataFrame to WGS84 coordinate reference system
    gdf_wgs84 = gdf.to_crs({'init': 'epsg:4326'})
    
    #Call Folium choropleth function, specifying the geometry as a the WGS84 dataframe converted to GeoJSON, the data as 
    #the GeoDataFrame, the columns as the user-specified id field and and value field.
    #key_on field refers to the id field within the GeoJSON string
    mapobj.choropleth(geo_str = gdf_wgs84.to_json(), data = gdf,
                columns = [id_field, value_field], key_on = 'feature.properties.{}'.format(id_field),
                fill_color = fill_color, fill_opacity = fill_opacity, line_opacity = line_opacity,  
                threshold_scale = threshold_scale)
    return mapobj

#Update basemap with choropleth
crime_map=add_choropleth(crime_map, tracts, 'CTFIPS10','AssaultsPSqMi')

### Create Crime Point Cluster Layer
Before displaying my map, I will also add the layer of crime point locations. Rather than display each individual point, I use Leaflet's marker clustering feature, which makes it easier to visualize large numbers of points by grouping together those that are close to each other. Additionally, I use popups to display information about a particular crime when the user clicks on a point. Folium lets you create HTML-rich popups called IFrames. I use this feature only in the most basic form, just to display a few lines of information: crime description, date, time, and address. There are obviously much more creative things that can be done with an IFrame popup (tables, graphs, sub-maps, etc.) but for my purposes this is all I need. 

My function takes as its inputs a Folium map object, a GeoDataFrame, and a list of fields to include in the popup. I run this function on my previously created map object (already updated with a choropleth layer), specifying 4 fields of interest that I want to display. 


In [ ]:
def add_point_clusters(mapobj, gdf, popup_field_list):
    #Create empty lists to contain the point coordinates and the point pop-up information
    coords, popups = [], [] 
    #Loop through each record in the GeoDataFrame
    for i, row in gdf.iterrows():
        #Append lat and long coordinates to "coords" list
        coords.append([row.geometry.y, row.geometry.x])
        #Create a string of HTML code used in the IFrame popup
        #Join together the fields in "popup_field_list" with a linebreak between them
        label = '<br>'.join([row[field] for field in popup_field_list])
        #Append an IFrame that uses the HTML string to the "popups" list 
        popups.append(IFrame(label, width = 300, height = 100))
        
    #Create a Folium feature group for this layer, since we will be displaying multiple layers
    pt_lyr = folium.FeatureGroup(name = 'pt_lyr')
    
    #Add the clustered points of crime locations and popups to this layer
    pt_lyr.add_children(MarkerCluster(locations = coords, popups = popups))
    
    #Add this point layer to the map object
    mapobj.add_children(pt_lyr)
    return mapobj

#Update choropleth with point clusters
crime_map = add_point_clusters(crime_map, assaults, ['Descript','Address','DateStr','Time'])

### Add Layer Control, Display and Save Map
As a finishing touch, I add Layer Control to the map, which allows me to toggle on/off either of my two layers (see widget on the top right). Then I save my finished map as an HTML and display it! 

I hope this was helpful in demonstrating some of the mapping capabilities of Leaflet accessed through the package Folium.  The functions I wrote provide a nice way of displaying two very common types of spatial data on a basemap and can obviously be tweaked to add more custom functionality.

In [ ]:
folium.LayerControl().add_to(crime_map) #Add layer control to toggle on/off
crime_map.save('sf_assaults.html') #save HTML
crime_map #display map